In [1]:
# Se importan las librerías
import pandas as pd
from neo4j import GraphDatabase
import time

In [2]:
# Se carga el csv final_distances de la carpeta output_modelo_parte2
# df = pd.read_json (r'C:\Users\migue\AplicacionesYTendencias\ProyectoAyT\UEM-Analytics-G22-3\BBDD-Grafos\final_distances.json')
# df.to_csv (r'C:\Users\migue\AplicacionesYTendencias\ProyectoAyT\UEM-Analytics-G22-3\BBDD-Grafos\final_distances.csv', index = None)

final_distances = pd.read_csv('final_distances.csv')

In [3]:
# Se carga el csv final_paper de la carpeta output_modelo_parte2
# df2 = pd.read_json (r'C:\Users\migue\AplicacionesYTendencias\ProyectoAyT\UEM-Analytics-G22-3\BBDD-Grafos\final_paper.json')
# df2.to_csv (r'C:\Users\migue\AplicacionesYTendencias\ProyectoAyT\UEM-Analytics-G22-3\BBDD-Grafos\final_paper.csv', index = None)

final_paper = pd.read_csv('final_paper.csv')

### Conectar a Neo4j Desktop

In [4]:
# Se crea una clase en la que se esctablece la conexión entre la máquina local donde
# se encuentra el código de python y la base de datos de Neo4j Desktop.
# Le indicamos la URL, el usuario y la contraseña para completar la conexión
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
        
conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="usermate",              
                       pwd="glass")

### Creación BBDD

In [5]:
# Llenamos la base de datos.
# Se crean CONSTRAINTS para garantizar que los nodos no estén
# duplicados mientras configuramos algunos índices.

conn.query('CREATE CONSTRAINT papers IF NOT EXISTS FOR (p:Paper) REQUIRE p.titulo IS UNIQUE')
conn.query('CREATE CONSTRAINT fuentes IF NOT EXISTS FOR (f:Fuente) REQUIRE f.fuente IS UNIQUE')
conn.query('CREATE CONSTRAINT autores IF NOT EXISTS FOR (a:Autor) REQUIRE a.autor IS UNIQUE')
conn.query('CREATE CONSTRAINT clases IF NOT EXISTS FOR (c:Clase) REQUIRE c.clase IS UNIQUE')
conn.query('CREATE CONSTRAINT keywords IF NOT EXISTS FOR (k:Keyword) REQUIRE k.keyword IS UNIQUE')

[]

In [6]:
# Se crea el nodo Paper con las propiedades de id, titulo, abstract y resumen.
# Con la sentencia UNWIND se separa cada valor de una lista en una fila.
# Se crea el nodo Clase con la propiedad de clase
# Se conectan los nodos de Paper y Clase mediante la relacion :EN_CLASE.

query = '''
//Create and Relation Papers and Categories
LOAD CSV WITH HEADERS FROM 'file:///final_paper.csv' AS row
MERGE (p:Paper {titulo:row.titulo}) ON CREATE SET p.abstract = row.abstract_limpio, p.resumen = row.resumen_limpio
WITH p, row
UNWIND split(row.clase_primaria, ', ') AS clase_primaria_name
MERGE (c:Clase {clase: clase_primaria_name})
MERGE (p)-[:EN_CLASE]->(c)
'''
conn.query(query,db='neo4j2')

[]

In [7]:
# Se crea el nodo Fuente con la propiedad de fuente.
# Se conectan los nodos de Paper y Fuente mediante la relacion :EN_FUENTE.

query = '''
//Create node for fuente and relationship for fuente and paper
WITH 'file:///final_paper.csv' AS data 
LOAD CSV WITH HEADERS FROM data AS row
MERGE (f:Fuente {fuente: row.fuente})
WITH row, f
MATCH (p:Paper {titulo: row.titulo})
MERGE (p)-[:EN_FUENTE]->(f)
'''
conn.query(query,db='neo4j2')

[]

In [8]:
# Se crea el nodo Autor con la propiedad de name.
# Con la sentencia UNWIND se separa cada autor de la lista de autores en una fila cada uno.
# Se conectan los nodos de Autor y Paper mediante la relacion :REALIZA.

query = '''
//Create node for Autor and Relationship between papers and autores
LOAD CSV WITH HEADERS FROM 'file:///final_paper.csv' AS row
UNWIND split(row.autores, ', ') AS autor_name
MERGE (a:Autor {name: autor_name})
WITH a, row
MATCH (p:Paper {titulo: row.titulo})
MERGE (a)-[:REALIZA]->(p)
'''
conn.query(query,db='neo4j2')

[]

In [9]:
# Se crea el nodo Keyword con la propiedad de keyword.
# Con la sentencia UNWIND se separa cada keyword de la lista de keywords en una fila cada uno.
# Se conectan los nodos de Paper y Keyword mediante la relacion :REALIZA.

query = '''
//Create node for Keywords and Relationship between papers and autores
LOAD CSV WITH HEADERS FROM 'file:///final_paper.csv' AS row
UNWIND split(row.keywords, ',') AS keyword_name
MERGE (k:Keyword {keyword: keyword_name})
WITH k, row
MATCH (p:Paper {titulo: row.titulo})
MERGE (p)-[:CONTIENE]->(k)
'''
conn.query(query,db='neo4j2')

[]

In [10]:
# Se conecta el nodo de Paper así mismo para obtener las distancias entre papers.

query = '''
//Create and Relation Papers and Categories
LOAD CSV WITH HEADERS FROM 'file:///final_distances.csv' AS row
MATCH (titulo_2:Paper {titulo: row.titulo_2})
MATCH (titulo:Paper {titulo: row.titulo})
MERGE (titulo_2)-[:DISTANCE {distance: toFloat(row.distance)}]->(titulo)
'''
conn.query(query,db='neo4j2')

[]